In [19]:
import numpy as np
from suppl import *
import random

### Overview 

Your model will have the following structure: 

- Initialize parameters 
- Run the optimization loop
    - Forward propagation to compute the loss function
    - Backward propagation to compute the gradients with respect to the loss function
    - Using the gradients, update your parameter with the gradient descent update rule.
- Return the learned parameters 
    
<img src="rnn.png" style="width:450;height:300px;">

In [20]:
## Reading the data
data = open('elements.txt', 'r').read()
data= data.lower()
print (data)
chars = list(set('\n'.join(data)))
print (chars)
data_size, vocab_size = len(data), len(chars)


hydrogen
helium
lithium
beryllium
boron
carbon
nitrogen
oxygen
fluorine
neon
sodium
magnesium
aluminum, aluminium
silicon
phosphorus
sulfur
chlorine
argon
potassium
calcium
scandium
titanium
vanadium
chromium
manganese
iron
cobalt
nickel
copper
zinc
gallium
germanium
arsenic
selenium
bromine
krypton
rubidium
strontium
yttrium
zirconium
niobium
molybdenum
technetium
ruthenium
rhodium
palladium
silver
cadmium
indium
tin
antimony
tellurium
iodine
xenon
cesium
barium
lanthanum
cerium
praseodymium
neodymium
promethium
samarium
europium
gadolinium
terbium
dysprosium
holmium
erbium
thulium
ytterbium
lutetium
hafnium
tantalum
tungsten
rhenium
osmium
iridium
platinum
gold
mercury
thallium
lead
bismuth
polonium
astatine
radon
francium
radium
actinium
thorium
protactinium
uranium
neptunium
plutonium
americium
curium
berkelium
californium
einsteinium
fermium
mendelevium
nobelium
lawrencium
rutherfordium
dubnium
seaborgium
bohrium
hassium
meitnerium
darmstadtium
roentgenium
copernicium
nihonium
fle

In [21]:
## Indexing the characters
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: ' ', 2: ',', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


## Sampling sequence
<img src="complete.png" style="width:500;height:300px;">

Run one step of forward propagation to get $a^{\langle 1 \rangle}$ and $\hat{y}^{\langle 1 \rangle}$. Here are the equations:

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

In [22]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [23]:
def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

In [24]:
def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

In [25]:
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

In [26]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [27]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    
    Returns:
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    """
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

In [28]:
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient,-maxValue,maxValue,gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


In [29]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    
    return a_next, p_t


In [30]:

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

In [31]:
def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

In [32]:
def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    # Initialize x, a and y_hat as empty dictionaries
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    # initialize your loss to 0
    loss = 0
    
    for t in range(len(X)):
        
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # Run one step forward of the RNN
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy term of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

In [33]:
def rnn_backward(X, Y, parameters, cache):
    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    ### START CODE HERE ###
    # Backpropagate through time
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    ### END CODE HERE ###
    gradients = clip(gradients,5)
    
    return gradients, a

In [34]:

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). 
    x = np.zeros((vocab_size,1))
    # Step 1a: Initialize a_prev as zeros 
    a_prev = np.zeros((n_a,1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate 
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevents entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)##Complete
        z = np.dot(Wya, a) + by##Complete
        y = softmax(z)##Complete
        
        
        np.random.seed(counter+seed) 
        
        # Step 3: Sampling the index of a character within the vocabulary from the probability distribution y [this part is already done]
        idx = np.random.choice(list(range(vocab_size)),p=y.ravel())

        # Append the index to "indices"
        indices.append(idx)##ADD THE LINE HERE
        
        # Step 4: Overwriting the input character as the one corresponding to the sampled index. [COMPLETED]
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a## ADD THE LINE HERE
        
        
        seed += 1
        counter +=1
        
    

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [35]:

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    ###Function definitions are provided in 'suppl.py', call the correct functions with correct arguments
    
    # Forward propagate through time 
    loss, cache = rnn_forward(X, Y, a_prev, parameters)## COMPLETE
    
    # Backpropagate through time 
    gradients, a = rnn_backward(X, Y, parameters, cache)## COMPLETE
    
    
    # Update parameters 
    parameters = update_parameters(parameters, gradients, learning_rate)## COMPLETE
    
    
    
    return loss, gradients, a[len(X)-1]

In [36]:

def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, element_names = 7, vocab_size = 27):
    """
    Trains the model and generates element names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    element_names -- number of element names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text, size of the vocabulary
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size## COMPLETE
    
    # Initialize parameters 
    parameters = initialize_parameters(n_a, n_x, n_y)## COMPLETE
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    loss = get_initial_loss(vocab_size, element_names)## COMPLETE
    
    # Build list of all element names (training examples).
    with open("elements.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    # Shuffle list of all element names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize 
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        
        # Use the hint above to define one training example (X,Y) 
        index = j % len(examples)
        X = [None]+ [char_to_ix[ch] for ch in examples[index]] ## Adding None to initialize a0 
        Y = X[1:] + [char_to_ix["\n"]]## COMPLETE ##  Y should start from the second element of X, i.e. if X='helium', we are trying to predict 'elium\n' [remember to add the (index of )newline (\n) character to denote end of word]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)## Complete
        
        
        
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters using sample() to check the model 
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(element_names):
                
                # Sample indices and print them
                sampled_indices =sample(parameters, char_to_ix, seed) ## COMPLETE ## call 'sample'
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  
      
            print('\n')
        
    return parameters

In [37]:
## After a few iterations you should see 'element-like' words generated (for example  ending with 'ium')
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.077451

Mizxwtbldpncyfspw shihvu
Imc,
Izxwtbldpncyfspw shihvu
Mc,
Zxwtbldpncyfspw shihvu
C,
Xwtbldpncyfspw shihvu


Iteration: 2000, Loss: 19.625536

Lhutprenerirumium
Ium
Iuprseolon
Labaertgacium
Wtrofinium
Ebaershbanumineium
Tpraneriun


Iteration: 4000, Loss: 16.298499

Ontusodium
Ium
Lutorium
Oldalum
Utopen
Eibhrpelbium
Torfium


Iteration: 6000, Loss: 13.721802

Postur
Mhlc
Mosrium
Pel
Tron
El
Tordium


Iteration: 8000, Loss: 12.046173

Orytium
Lobabium
Luton
Olabium
Trosepmirovium
El
Toren


Iteration: 10000, Loss: 11.361334

Oronesmicon
Lutebium
Lutonium
Omadeneeldium
Trosesenesserium
Efberkercium
Troditercorium


Iteration: 12000, Loss: 11.000204

Orytnium
Holc
Ium
Old
Trrium
Elberium
Tromium


Iteration: 14000, Loss: 9.637780

Oryttium
Ium
Luton
Old
Tutrium
Elenium
Tuthenium


Iteration: 16000, Loss: 9.466895

Orytrinium
Lutachorium
Luton
Ordanium
Tutrium
Eicium
Tuthenium


Iteration: 18000, Loss: 11.706154

Oltthonium
Lutbalsthalminium
Luton
Oladium
Tut